<a href="https://colab.research.google.com/github/OtsukiTaisuke/Colab/blob/master/NA2_CNN_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 概要

## NNを使ったmnistデータ識別

1. データの収集・整理：mnistデータセット
1. 前処理　　　　　　：最大値の調整
1. 評価法の設定　　　：分割法（訓練用とテスト用）
1. 学習　　　　　　　：DNN
1. 結果の可視化　　　：正答率（accuracy）

# 前準備


## 各パッケージのインポート
1. numpy: 行列計算用
1. TensorFlowなどの機械学習ライブラリを楽に扱うライブラリ

In [0]:
import numpy as np
import keras

# データセットの読み込み

## mnistデータセットの読み込み

+ 読み込んだデータセットをそれぞれ  
訓練に用いるデータ(X_train)＆正解ラベル(y_train)  
テストに用いるデータ(X_test)＆正解ラベル(y_test)  
に代入
+ 学習用データが60000枚，評価用が10000枚ある

In [0]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# データの整理，前処理

## 入力データ（X_train, X_test）の処理

+ 使うデータ数を制限（訓練に10000枚，テストに1000枚）
+ 画素ごとのデータの値が0～255→0～1に変換

In [0]:
img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows,  img_cols, 1)[0:10000]
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)[0:1000]

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

## ラベルデータ（y_train, y_test）の処理

+ 正解ラベル（y_train, y_test）をone-hotベクトル表現

In [0]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train)[0:10000]
Y_test = np_utils.to_categorical(y_test)[0:1000]

# 多階層ニューラルネットワークの構成

## パッケージ読み込み

+ Sequential：階層状のネットワークの構成
+ Dense：ネットワークレイヤー
+ Activation：活性化関数の設定
+ SGD：確率的最急勾配法

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD

## 学習モデルの構築
### 構築するモデルのサイズの設定

+ 入力のサイズを訓練データから取得
+ 隠れ層のノード数を200に設定
+ 出力のサイズをラベルデータから取得

In [0]:
input_shape = X_train[0].shape
n_hidden = 200
n_out = len(Y_train[0])

### CNN
- 畳み込み
- プーリング
- 畳み込み
- プーリング
- 全結合層

In [0]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu',
                input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
                input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(n_out, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               102528    
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1290      
Total para

### モデルのコンパイル

+ 損失関数：クロスエントロピー
+ 最適化アルゴリズム：SGD
+ 評価基準：正答率（accuracy）

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.1),
             metrics = ['accuracy'])

# 学習

## fitメソッドによる学習

+ epochs：繰り返し回数
+ batch_size：バッチサイズ

In [0]:
model.fit(X_train, Y_train, epochs=5, batch_size=200)

Epoch 1/5
10000/10000 [==============================] - 5s 532us/step - loss: 1.5397 - acc: 0.5309
Epoch 2/5
10000/10000 [==============================] - 5s 500us/step - loss: 0.4692 - acc: 0.8548
Epoch 3/5
10000/10000 [==============================] - 5s 496us/step - loss: 0.2569 - acc: 0.9272
Epoch 4/5
10000/10000 [==============================] - 5s 497us/step - loss: 0.2059 - acc: 0.9389
Epoch 5/5
10000/10000 [==============================] - 5s 504us/step - loss: 0.1743 - acc: 0.9481


# 結果の可視化

## evaluateメソッドによる評価

+ 損失値（Loss）と正答率（accuracy）を表示

In [0]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.15894902217388154
Test accuracy: 0.949
